In [15]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from openai import OpenAI
import pandas as pd
import numpy as np
import json

In [17]:
dataset = pd.read_csv('dataset/search_keywords.csv')
print(dataset.head(5))

   id                                             prompt  user_id  tenant_id  \
0   1  Can you filter out all tasks that have been ma...  23456.0      456.0   
1   2  I need a list of tasks that are escalated and ...      NaN        NaN   
2   3  Show me all tasks that are marked as approved ...  11223.0        NaN   
3   4  Find tasks with the type 'input' and created b...  45678.0      123.0   
4   5  Locate tasks with the workflow node name 'Appr...      NaN        NaN   

  user_group_id workflow_id    card_id label_id label_name workflow_node_name  \
0           NaN         NaN        NaN      NaN        NaN                NaN   
1      group987         NaN        NaN      NaN     Urgent                NaN   
2           NaN     card123        NaN   Review        NaN           approved   
3           NaN         NaN        NaN      NaN        NaN              input   
4           NaN         NaN  parent456      NaN        NaN           Approval   

      status    type           c

In [22]:
def extract_data_from_keyword(keyword):
    # Load system content and user content
    system_content = load_prompt('prompts/search_keywords_system_prompt.txt')
    
    user_content = f"Keywords: {keyword}"

    # Initialize the OpenAI API client
    client = OpenAI()

    # Make a request to the OpenAI API to generate a chat completion
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.3,
        max_tokens=4096,
        top_p=0.8,
        response_format={ "type": "json_object" }
    )

    # Extract the completion result and token usage information from the response
    completion = chat_completion.choices[0].message.content
    result = json.loads(completion)
    prompt_tokens_used = chat_completion.usage.prompt_tokens
    completion_tokens_used = chat_completion.usage.completion_tokens

    return result, prompt_tokens_used, completion_tokens_used

In [20]:
def load_prompt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [21]:
keyword = "Show me all tasks that are marked as approved and were created by user ID 11223. Additionally, they should have a due date in the next week, contain the label name 'Review', and be associated with card ID 'card123'."
result, prompt_tokens_used, completion_tokens_used = extract_data_from_keyword(keyword)

print(f"Result : {result}")
print(f"Prompt Tokens Used : {prompt_tokens_used}")
print(f"Completion Tokens Used : {completion_tokens_used}")

You are an intelligent assistant tasked with filtering and extracting information from user inputs on a form submission platform. Each input will be a combination of various field names and values. Extract and format the information as follows:

- `id`: Unique identifier for the task.
- `title`: Title of the task.
- `type`: Type of the task.
- `card_id`: Unique identifier of the associated card.
- `due_date`: Due date of the task.
- `created_date`: Date the task was created.
- `completed_date`: Date the task was completed.
- `status`: Status of the task.
- `node_id`: Unique identifier for the workflow node associated with the task.
- `node_name`: Name of the workflow node associated with the task.
- `comment`: Comment added by the task assignee.
- `label_id`: Identifier for the label associated with the workflow node.
- `label_name`: Name of the label associated with the workflow node.
- `escalated`: Indicates if the task is the result of an escalation.
- `processed`: Indicates if the 